In [1]:
import os
import pandas as pd
from google.colab import auth
from datetime import datetime

In [2]:
auth.authenticate_user()

In [3]:
!unzip /content/flow-forecast.zip

Archive:  /content/flow-forecast.zip
d37ed04c27cc14b7dc5a66af001255b1a1bb99c7
   creating: flow-forecast-master/
   creating: flow-forecast-master/.circleci/
  inflating: flow-forecast-master/.circleci/config.yml  
 extracting: flow-forecast-master/.flake8  
   creating: flow-forecast-master/.github/
  inflating: flow-forecast-master/.github/dependabot.yml  
   creating: flow-forecast-master/.github/workflows/
  inflating: flow-forecast-master/.github/workflows/python-publish.yml  
  inflating: flow-forecast-master/.gitignore  
  inflating: flow-forecast-master/LICENSE  
  inflating: flow-forecast-master/README.md  
   creating: flow-forecast-master/docs/
  inflating: flow-forecast-master/docs/Makefile  
  inflating: flow-forecast-master/docs/make.bat  
  inflating: flow-forecast-master/docs/requirements.txt  
   creating: flow-forecast-master/docs/source/
  inflating: flow-forecast-master/docs/source/basic_ae.rst  
  inflating: flow-forecast-master/docs/source/basic_utils.rst  
  infl

In [3]:
os.chdir('/content/github_aistream-peelout_flow-forecast')

In [4]:
!pip install -r requirements.txt

In [5]:
!pip install git+https://github.com/CoronaWhy/task-geo.git

  Cloning https://github.com/CoronaWhy/task-geo.git to /tmp/pip-req-build-pmzyxatz
  Running command git clone -q https://github.com/CoronaWhy/task-geo.git /tmp/pip-req-build-pmzyxatz


In [6]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [7]:
from flood_forecast.trainer import train_function

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [8]:
!pip install wandb

In [9]:
!wandb login

wandb: Currently logged in as: sakil786 (use `wandb login --relogin` to force relogin)


In [10]:
# Get wind data
!wget https://storage.googleapis.com/coronaviruspublicdata/forecast_2/wind.csv
!wget https://storage.googleapis.com/coronaviruspublicdata/forecast_2/solar.csv

--2021-08-16 17:51:21--  https://storage.googleapis.com/coronaviruspublicdata/forecast_2/wind.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 2607:f8b0:4023:c0b::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4936771 (4.7M) [text/csv]
Saving to: ‘wind.csv.1’

wind.csv.1          100%[===================>]   4.71M  --.-KB/s    in 0.03s   

2021-08-16 17:51:21 (178 MB/s) - ‘wind.csv.1’ saved [4936771/4936771]

--2021-08-16 17:51:21--  https://storage.googleapis.com/coronaviruspublicdata/forecast_2/solar.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 2607:f8b0:4023:c0b::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6156494 (5.9M) [text/csv]
Saving to: ‘solar.csv.1’

so

In [11]:
wind1 = pd.read_csv('wind.csv')

In [12]:
wind1.head()

,time,AT,BE,BG,CH,CZ,DE,DK,EE,ES,FI,FR,EL,HR,HU,IE,IT,LT,LU,LV,NL,NO,PL,PT,RO,SI,SK,SE,UK
0,1986-01-01 00:00:00,0.047786,0.023020,0.048940,0.065907,0.041685,0.031583,0.017365,0.014149,0.079043,0.005800,0.049822,0.051933,0.030507,0.025005,0.011889,0.046250,0.035880,0.014839,0.019004,0.014293,0.010351,0.029919,0.076675,0.029107,0.015193,0.054001,0.017463,0.030419
1,1986-01-02 00:00:00,0.045921,0.036297,0.067995,0.077502,0.026427,0.023506,0.014981,0.015682,0.119019,0.007176,0.063090,0.115133,0.035716,0.039431,0.016575,0.051848,0.016988,0.019510,0.013771,0.020373,0.006469,0.031359,0.106900,0.044379,0.024623,0.034362,0.008086,0.022146
2,1986-01-03 00:00:00,0.067308,0.021352,0.101287,0.103680,0.057274,0.046181,0.023478,0.009570,0.106574,0.004687,0.048678,0.123855,0.051901,0.029249,0.054734,0.062773,0.017350,0.056217,0.011871,0.010782,0.007217,0.027554,0.160308,0.047235,0.032093,0.023788,0.010004,0.060345
3,1986-01-04 00:00:00,0.043833,0.050756,0.039337,0.075418,0.025843,0.025011,0.020003,0.008595,0.135060,0.004102,0.092991,0.089767,0.055547,0.044340,0.016779,0.055305,0.019638,0.055925,0.013604,0.030366,0.007998,0.025986,0.208236,0.037510,0.028663,0.018115,0.009546,0.030981
4,1986-01-05 00:00:00,0.082394,0.014302,0.033055,0.090867,0.065186,0.028168,0.016261,0.009780,0.095232,0.005172,0.045049,0.074312,0.081576,0.082401,0.038972,0.102499,0.020079,0.018873,0.013913,0.012728,0.007241,0.047764,0.115451,0.037254,0.057101,0.072843,0.013872,0.023346


In [13]:
wind=pd.DataFrame(wind1,columns={"time","AT"})

In [14]:
wind.head()

,AT,time
0,0.047786,1986-01-01 00:00:00
1,0.045921,1986-01-02 00:00:00
2,0.067308,1986-01-03 00:00:00
3,0.043833,1986-01-04 00:00:00
4,0.082394,1986-01-05 00:00:00


In [15]:
wind['datetime'] = pd.to_datetime(wind['time']).dt.date
wind.set_index('datetime', drop=False)
wind.head()

,AT,time,datetime
0,0.047786,1986-01-01 00:00:00,1986-01-01
1,0.045921,1986-01-02 00:00:00,1986-01-02
2,0.067308,1986-01-03 00:00:00,1986-01-03
3,0.043833,1986-01-04 00:00:00,1986-01-04
4,0.082394,1986-01-05 00:00:00,1986-01-05


**Pre training on Wind Data**

In [16]:
import pandas as pd

wind['datetime'] = pd.to_datetime(wind['time']).dt.date
wind.set_index('datetime', drop=False)
wind.head()

,AT,time,datetime
0,0.047786,1986-01-01 00:00:00,1986-01-01
1,0.045921,1986-01-02 00:00:00,1986-01-02
2,0.067308,1986-01-03 00:00:00,1986-01-03
3,0.043833,1986-01-04 00:00:00,1986-01-04
4,0.082394,1986-01-05 00:00:00,1986-01-05


In [17]:
!pip install pycountry

In [18]:
import pycountry
names = {}
for code in wind.columns:
    try:
        names[code] = pycountry.countries.get(alpha_2=code).name
    except:
        print(code)

time
datetime


In [19]:
wind.rename(columns = names, inplace=True)
# wind['year'] = pd.to_datetime(wind['time']).map(lambda x: x.year)
wind['month'] = pd.to_datetime(wind['time']).map(lambda x: x.month)
wind['weekday'] = pd.to_datetime(wind['time']).map(lambda x: x.weekday())

In [20]:
wind.head()

,Austria,time,datetime,month,weekday
0,0.047786,1986-01-01 00:00:00,1986-01-01,1,2
1,0.045921,1986-01-02 00:00:00,1986-01-02,1,3
2,0.067308,1986-01-03 00:00:00,1986-01-03,1,4
3,0.043833,1986-01-04 00:00:00,1986-01-04,1,5
4,0.082394,1986-01-05 00:00:00,1986-01-05,1,6


In [21]:
# Making seperate dataframes for each country's data and saving in seperate CSV files
!mkdir wind
country_wise = {}
for country in names.values():
    country_wise[country] = wind[['datetime', 'month', 'weekday', country]]
    country_wise[country].to_csv('wind/'+country+'.csv')
country_wise[list(country_wise.keys())[0]].head()

mkdir: cannot create directory ‘wind’: File exists


,datetime,month,weekday,Austria
0,1986-01-01,1,2,0.047786
1,1986-01-02,1,3,0.045921
2,1986-01-03,1,4,0.067308
3,1986-01-04,1,5,0.043833
4,1986-01-05,1,6,0.082394


In [22]:
wind.tail()

,Austria,time,datetime,month,weekday
10952,0.119231,2015-12-27 00:00:00,2015-12-27,12,6
10953,0.128963,2015-12-28 00:00:00,2015-12-28,12,0
10954,0.094872,2015-12-29 00:00:00,2015-12-29,12,1
10955,0.086713,2015-12-30 00:00:00,2015-12-30,12,2
10956,0.111772,2015-12-31 00:00:00,2015-12-31,12,3


In [23]:
# Config file for WanDB sweeps

def make_config_file1(file_path, df_len):
  run = wandb.init(project="pretrained-wind-updatedfinal")
  wandb_config = wandb.config
  train_number = df_len * .7
  validation_number = df_len *.9
  config_default={                 
    "model_name": "MultiAttnHeadSimple",
    "model_type": "PyTorch",
    "model_params": {
      "number_time_series": 3,
      "seq_len":wandb_config["forecast_history"], 
      "output_seq_len":wandb_config["out_seq_length"],
      "forecast_length":wandb_config["out_seq_length"]
     },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"],
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "target_col": [file_path.split('.')[0].split('/')[1]],
       "relevant_cols": [file_path.split('.')[0].split('/')[1], "month", "weekday"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "GCS": False,
    
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE"],
   "inference_params":
   {     
         "datetime_start":"2010-01-01",
          "hours_to_forecast": 50, 
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"],
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": [file_path.split('.')[0].split('/')[1], "month", "weekday"],
             "target_col": [file_path.split('.')[0].split('/')[1]],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  wandb.config.update(config_default)
  return config_default

sweep_config = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [2]
        },
        "lr":{
            "values":[0.001]
        },
        "forecast_history":{
            "values":[1, 2]
        },
        "out_seq_length":{
            "values":[1, 2]
        }
    }
}

In [24]:
import os
import pandas as pd

# The countries we will be pretraining our wind data on
os.listdir('wind')

['Austria.csv']

In [25]:
# This definitely works
import wandb
for country in os.listdir('wind'):
    file_path = 'wind/'+country
    full_len = len(pd.read_csv(file_path))
    sweep_id = wandb.sweep(sweep_config, project="pretrained-wind-updatedfinal")
    wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file1(file_path, full_len)))
    #!gsutil cp -n -r model_save gs://coronaviruspublicdata/pretrained

Create sweep with ID: xynsw8b3
Sweep URL: https://wandb.ai/sakil786/pretrained-wind-updatedfinal/sweeps/xynsw8b3


wandb: Agent Starting Run: htbbwtha with config:
wandb: 	batch_size: 2
wandb: 	forecast_history: 1
wandb: 	lr: 0.001
wandb: 	out_seq_length: 1
wandb: Currently logged in as: sakil786 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


interpolate should be below
[]
Now loading wind/Austria.csv
scaling now
interpolate should be below
[]
Now loading wind/Austria.csv
scaling now
interpolate should be below
[]
Now loading wind/Austria.csv
scaling now
Using Wandb config:
{'batch_size': 2, 'forecast_history': 1, 'lr': 0.001, 'out_seq_length': 1, 'model_name': 'MultiAttnHeadSimple', 'model_type': 'PyTorch', 'model_params': {'number_time_series': 3, 'seq_len': 1, 'output_seq_len': 1, 'forecast_length': 1}, 'dataset_params': {'class': 'default', 'training_path': 'wind/Austria.csv', 'validation_path': 'wind/Austria.csv', 'test_path': 'wind/Austria.csv', 'batch_size': 2, 'forecast_history': 1, 'forecast_length': 1, 'train_end': 7669, 'valid_start': 7670, 'valid_end': 9861, 'target_col': ['Austria'], 'relevant_cols': ['Austria', 'month', 'weekday'], 'scaler': 'StandardScaler', 'interpolate': False}, 'training_params': {'criterion': 'MSE', 'optimizer': 'Adam', 'optim_params': {}, 'lr': 0.001, 'epochs': 10, 'batch_size': 2}, 'GCS

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.



Now plotting final plots


...

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.



Now plotting final plots


epoch,9
_runtime,273
_timestamp,1629137558
_step,36
loss,0.54343


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
_runtime,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▆▆▇▇█████████████████
_timestamp,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▆▆▇▇█████████████████
_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▁▂▂▁▂▁▁▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
# #saving model
# torch.save(model.state_dict(), 'checkpoint.pth')

In [39]:
# # It gave some errors, maybe some encoding ones.
# import wandb
# !mkdir model_save
# for country in os.listdir('wind'):
#     file_path = 'wind/'+country
#     full_len = len(pd.read_csv(file_path))
#     sweep_id = wandb.sweep(sweep_config, project="pretrained-wind-updatedfinal")
#     paths = []
#     if len(os.listdir("model_save"))>1:
#         print("will use transfer")
#         weight_files = filter(lambda x: x.endswith(".pth"), os.listdir("model_save")) 
#         for weight_file in weight_files:
#           paths.append(os.path.join("model_save", weight_file)) 
#         correct_file = max(paths, key = os.path.getctime)
#         print(correct_file) 
#         wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file1(correct_file, full_len)))
#     else:
#         wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file1(file_path, full_len)))
#     print("sucessfully completed sweep for: " + file_path)
#    # !gsutil cp -n -r model_save gs://coronaviruspublicdata/pretrained

mkdir: cannot create directory ‘model_save’: File exists
Create sweep with ID: obx2fbxx
Sweep URL: https://wandb.ai/sakil786/pretrained-wind-updatedfinal/sweeps/obx2fbxx
will use transfer
model_save/16_August_202106_44PM_model.pth


wandb: Agent Starting Run: se1ls2bg with config:
wandb: 	batch_size: 2
wandb: 	forecast_history: 1
wandb: 	lr: 0.001
wandb: 	out_seq_length: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


interpolate should be below


Run se1ls2bg errored: ParserError('Error tokenizing data. C error: Expected 1 fields in line 9, saw 2\n')
wandb: ERROR Run se1ls2bg errored: ParserError('Error tokenizing data. C error: Expected 1 fields in line 9, saw 2\n')
wandb: Agent Starting Run: xh9hgimn with config:
wandb: 	batch_size: 2
wandb: 	forecast_history: 1
wandb: 	lr: 0.001
wandb: 	out_seq_length: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


interpolate should be below


Run xh9hgimn errored: ParserError('Error tokenizing data. C error: Expected 1 fields in line 9, saw 2\n')
wandb: ERROR Run xh9hgimn errored: ParserError('Error tokenizing data. C error: Expected 1 fields in line 9, saw 2\n')
wandb: Agent Starting Run: omy255l9 with config:
wandb: 	batch_size: 2
wandb: 	forecast_history: 1
wandb: 	lr: 0.001
wandb: 	out_seq_length: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


interpolate should be below


Run omy255l9 errored: ParserError('Error tokenizing data. C error: Expected 1 fields in line 9, saw 2\n')
wandb: ERROR Run omy255l9 errored: ParserError('Error tokenizing data. C error: Expected 1 fields in line 9, saw 2\n')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


sucessfully completed sweep for: wind/Austria.csv


# **Transfer Learning**

# **Tranasfer learning on Solar Data**

In [26]:
import pandas as pd
solar = pd.read_csv('solar.csv')

In [27]:
solar.head()

,LocalTime,Power(MW),Power(MW).1,Power(MW).2,Power(MW).3,Power(MW).4,Power(MW).5,Power(MW).6,Power(MW).7,Power(MW).8,Power(MW).9,Power(MW).10,Power(MW).11,Power(MW).12,Power(MW).13,Power(MW).14,Power(MW).15,Power(MW).16,Power(MW).17,Power(MW).18,Power(MW).19,Power(MW).20,Power(MW).21,Power(MW).22,Power(MW).23,Power(MW).24,Power(MW).25,Power(MW).26,Power(MW).27,Power(MW).28,Power(MW).29,Power(MW).30,Power(MW).31,Power(MW).32,Power(MW).33,Power(MW).34,Power(MW).35,Power(MW).36,Power(MW).37,Power(MW).38,...,Power(MW).97,Power(MW).98,Power(MW).99,Power(MW).100,Power(MW).101,Power(MW).102,Power(MW).103,Power(MW).104,Power(MW).105,Power(MW).106,Power(MW).107,Power(MW).108,Power(MW).109,Power(MW).110,Power(MW).111,Power(MW).112,Power(MW).113,Power(MW).114,Power(MW).115,Power(MW).116,Power(MW).117,Power(MW).118,Power(MW).119,Power(MW).120,Power(MW).121,Power(MW).122,Power(MW).123,Power(MW).124,Power(MW).125,Power(MW).126,Power(MW).127,Power(MW).128,Power(MW).129,Power(MW).130,Power(MW).131,Power(MW).132,Power(MW).133,Power(MW).134,Power(MW).135,Power(MW).136
0,2006-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2006-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2006-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2006-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2006-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
solar.describe()

,Power(MW),Power(MW).1,Power(MW).2,Power(MW).3,Power(MW).4,Power(MW).5,Power(MW).6,Power(MW).7,Power(MW).8,Power(MW).9,Power(MW).10,Power(MW).11,Power(MW).12,Power(MW).13,Power(MW).14,Power(MW).15,Power(MW).16,Power(MW).17,Power(MW).18,Power(MW).19,Power(MW).20,Power(MW).21,Power(MW).22,Power(MW).23,Power(MW).24,Power(MW).25,Power(MW).26,Power(MW).27,Power(MW).28,Power(MW).29,Power(MW).30,Power(MW).31,Power(MW).32,Power(MW).33,Power(MW).34,Power(MW).35,Power(MW).36,Power(MW).37,Power(MW).38,Power(MW).39,...,Power(MW).97,Power(MW).98,Power(MW).99,Power(MW).100,Power(MW).101,Power(MW).102,Power(MW).103,Power(MW).104,Power(MW).105,Power(MW).106,Power(MW).107,Power(MW).108,Power(MW).109,Power(MW).110,Power(MW).111,Power(MW).112,Power(MW).113,Power(MW).114,Power(MW).115,Power(MW).116,Power(MW).117,Power(MW).118,Power(MW).119,Power(MW).120,Power(MW).121,Power(MW).122,Power(MW).123,Power(MW).124,Power(MW).125,Power(MW).126,Power(MW).127,Power(MW).128,Power(MW).129,Power(MW).130,Power(MW).131,Power(MW).132,Power(MW).133,Power(MW).134,Power(MW).135,Power(MW).136
count,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,...,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.00000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000,5832.000000
mean,97.990021,72.601543,72.617678,181.308368,163.551646,64.378909,68.242027,118.852452,79.612140,101.382236,75.003927,77.286043,69.262551,78.501012,73.466770,79.956670,78.386968,73.687637,72.940055,73.225892,77.151800,77.681207,77.547582,77.543021,76.565895,76.831962,76.478326,78.114215,77.010734,70.163134,77.808933,70.796005,70.287329,77.684585,76.008676,76.927332,77.182888,72.178669,77.444770,77.269050,...,73.652795,69.244050,72.905161,177.831687,64.941169,73.224571,54.165501,143.012654,78.924091,69.42296,68.952023,54.896039,75.234688,53.857613,64.792473,61.921416,72.592370,54.449314,72.350617,77.925634,72.908951,151.620405,41.334774,77.789300,70.294016,75.423577,41.880676,76.978515,163.736540,76.747531,77.040929,76.639506,123.436848,77.488529,76.994170,123.925892,75.624348,72.612363,61.697565,74.114180
std,121.215824,100.775117,102.617019,255.577980,227.711359,91.088087,96.013131,167.768777,109.091165,142.448961,105.413903,106.748239,96.573719,108.095548,103.759917,109.660217,111.123343,104.098793,103.392740,103.677634,108.577382,108.463883,108.651308,106.827237,105.974013,107.632612,105.833609,110.312326,107.852568,98.031765,109.073624,96.789302,97.835354,108.632086,106.472926,107.922032,106.690069,102.596689,107.988948,107.989447,...,92.000652,97.306594,100.894989,216.932139,91.735845,103.566030,75.384950,198.830916,108.327914,96.93925,96.811196,76.699214,105.744341,74.698423,91.477063,85.882681,100.702548,76.392924,101.918631,109.124838,103.196428,186.609687,57.095260,109.048518,96.184135,106.200268,57.693641,108.015106,228.709291,106.271689,107.825797,105.532143,170.698819,109.003895,107.316424,170.578983,93.098493,103.116330,86.162491,104.574143
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [29]:
solar.tail()

,LocalTime,Power(MW),Power(MW).1,Power(MW).2,Power(MW).3,Power(MW).4,Power(MW).5,Power(MW).6,Power(MW).7,Power(MW).8,Power(MW).9,Power(MW).10,Power(MW).11,Power(MW).12,Power(MW).13,Power(MW).14,Power(MW).15,Power(MW).16,Power(MW).17,Power(MW).18,Power(MW).19,Power(MW).20,Power(MW).21,Power(MW).22,Power(MW).23,Power(MW).24,Power(MW).25,Power(MW).26,Power(MW).27,Power(MW).28,Power(MW).29,Power(MW).30,Power(MW).31,Power(MW).32,Power(MW).33,Power(MW).34,Power(MW).35,Power(MW).36,Power(MW).37,Power(MW).38,...,Power(MW).97,Power(MW).98,Power(MW).99,Power(MW).100,Power(MW).101,Power(MW).102,Power(MW).103,Power(MW).104,Power(MW).105,Power(MW).106,Power(MW).107,Power(MW).108,Power(MW).109,Power(MW).110,Power(MW).111,Power(MW).112,Power(MW).113,Power(MW).114,Power(MW).115,Power(MW).116,Power(MW).117,Power(MW).118,Power(MW).119,Power(MW).120,Power(MW).121,Power(MW).122,Power(MW).123,Power(MW).124,Power(MW).125,Power(MW).126,Power(MW).127,Power(MW).128,Power(MW).129,Power(MW).130,Power(MW).131,Power(MW).132,Power(MW).133,Power(MW).134,Power(MW).135,Power(MW).136
5827,2006-08-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5828,2006-08-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5829,2006-08-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5830,2006-08-31 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5831,2006-08-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
solar.rename(columns={'LocalTime':'datetime'}, inplace=True)
solar = solar[['datetime', 'Power(MW)']]
solar.set_index('datetime', drop=False)

,datetime,Power(MW)
datetime,,
2006-01-01 00:00:00,2006-01-01 00:00:00,0.0
2006-01-01 01:00:00,2006-01-01 01:00:00,0.0
2006-01-01 02:00:00,2006-01-01 02:00:00,0.0
2006-01-01 03:00:00,2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,2006-01-01 04:00:00,0.0
...,...,...
2006-08-31 19:00:00,2006-08-31 19:00:00,0.0
2006-08-31 20:00:00,2006-08-31 20:00:00,0.0
2006-08-31 21:00:00,2006-08-31 21:00:00,0.0


In [31]:
solar = solar.iloc[1470:5826]
solar.describe()

,Power(MW)
count,4356.000000
mean,108.773531
std,125.396285
min,0.000000
25%,0.000000
50%,29.450000
75%,231.725000
max,371.100000


In [32]:
# Extracting the timeseries relevant columns
# solar['day'] = pd.to_datetime(solar['datetime']).map(lambda x: x.day)
solar['month'] = pd.to_datetime(solar['datetime']).map(lambda x: x.month)
solar['weekday'] = pd.to_datetime(solar['datetime']).map(lambda x: x.weekday())
# solar['hour'] = pd.to_datetime(solar['datetime']).map(lambda x: x.hour)

In [33]:
solar.to_csv('selected_solar.csv')
solar.tail()

,datetime,Power(MW),month,weekday
5821,2006-08-31 13:00:00,111.3,8,3
5822,2006-08-31 14:00:00,201.1,8,3
5823,2006-08-31 15:00:00,203.8,8,3
5824,2006-08-31 16:00:00,135.5,8,3
5825,2006-08-31 17:00:00,52.9,8,3


In [51]:
# # Config file for WanDB sweeps

# def make_config_file(file_path, df_len,weight_path=None):
#   run = wandb.init(project="pretrained-solar-updated")
#   wandb_config = wandb.config
#   train_number = df_len * .7
#   validation_number = df_len *.9
#   config_default={                 
#     "model_name": "MultiAttnHeadSimple",
#     "model_type": "PyTorch",
#     "model_params": {
#       "number_time_series":3,
#       "seq_len":wandb_config["forecast_history"], 
#       "output_seq_len":wandb_config["out_seq_length"],
#       "forecast_length":wandb_config["out_seq_length"]
#      },
#     "dataset_params":
#     {  "class": "default",
#        "training_path": file_path,
#        "validation_path": file_path,
#        "test_path": file_path,
#        "batch_size":wandb_config["batch_size"],
#        "forecast_history":wandb_config["forecast_history"],
#        "forecast_length":wandb_config["out_seq_length"],
#        "train_end": int(train_number),
#        "valid_start":int(train_number+1),
#        "valid_end": int(validation_number),
#        "target_col": ["Power(MW)"],
#        "relevant_cols": ["Power(MW)", "month", "weekday"],
#        "scaler": "StandardScaler", 
#        "interpolate": False
#     },
#     "training_params":
#     {
#        "criterion":"MSE",
#        "optimizer": "Adam",
#        "optim_params":
#        {

#        },
#        "lr": wandb_config["lr"],
#        "epochs": 10,
#        "batch_size":wandb_config["batch_size"]
    
#     },
#     "GCS": False,
    
#     "sweep":True,
#     "wandb":False,
#     "forward_params":{},
#    "metrics":["MSE"],
#    "inference_params":
#    {     
#          "datetime_start":"2006-08-22",
#           "hours_to_forecast":150, 
#           "test_csv_path":file_path,
#           "decoder_params":{
#               "decoder_function": "simple_decode", 
#             "unsqueeze_dim": 1
#           },
#           "dataset_params":{
#              "file_path": file_path,
#              "forecast_history":wandb_config["forecast_history"],
#              "forecast_length":wandb_config["out_seq_length"],
#              "relevant_cols": ["Power(MW)", "month", "weekday"],
#              "target_col": ["Power(MW)"],
#              "scaling": "StandardScaler",
#              "interpolate_param": False
#           }
#       }
#   }
#   if weight_path: 
#     config_default["weight_path"] = weight_path
#   wandb.config.update(config_default)
#   return config_default
#   # wandb.config.update(config_default)
#   # return config_default

# sweep_config = {
#   "name": "Default sweep",
#   "method": "grid",
#   "parameters": {
#         "batch_size": {
#             "values": [2]
#         },
#         "lr":{
#             "values":[0.001]
#         },
#         "forecast_history":{
#             "values":[1, 2]
#         },
#         "out_seq_length":{
#             "values":[1, 2]
#         }
#     }
# }

In [37]:
def make_config_file(file_path, df_len, weight_path=None):
  run = wandb.init(project="pretrain-counties")
  wandb_config = wandb.config
  train_number = df_len * .7
  validation_number = df_len *.9
  config_default={                 
    "model_name": "MultiAttnHeadSimple",
    "model_type": "PyTorch",
    "model_params": {
      "number_time_series":3,
      "seq_len":wandb_config["forecast_history"], 
      "output_seq_len":wandb_config["out_seq_length"],
      "forecast_length":wandb_config["out_seq_length"]
     },
     "weight_path_add":{
         "excluded_layers":["last_layer.weight", "last_layer.bias"]
     },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"],
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "target_col": ["Power(MW)"],
       "relevant_cols": ["Power(MW)", "month", "weekday"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "GCS": False,
    
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE"],
   "inference_params":
   {     
         "datetime_start":"2020-04-21",
          "hours_to_forecast":10, 
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"],
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": ["Power(MW)", "month", "weekday"],
             "target_col": ["Power(MW)"],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  if weight_path: 
    config_default["weight_path"] = weight_path
  wandb.config.update(config_default)
  return config_default

sweep_config = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [2, 3]
        },
        "lr":{
            "values":[0.001, 0.01]
        },
        "forecast_history":{
            "values":[1, 2, 3]
        },
        "out_seq_length":{
            "values":[1, 2, 3]
        }
    }
}

In [38]:
# Running the sweep
import wandb
file_path = 'selected_solar.csv'
full_len = len(pd.read_csv(file_path))
sweep_id = wandb.sweep(sweep_config, project="pretrain-counties")
wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file(file_path, full_len,weight_path="/content/github_aistream-peelout_flow-forecast/model_save/16_August_202106_12PM_model.pth")))

Create sweep with ID: 8ctw1vwh
Sweep URL: https://wandb.ai/sakil786/pretrain-counties/sweeps/8ctw1vwh


wandb: Agent Starting Run: 03hm68iz with config:
wandb: 	batch_size: 2
wandb: 	forecast_history: 1
wandb: 	lr: 0.001
wandb: 	out_seq_length: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


sucessfully deleted layers
Weights sucessfully loaded
interpolate should be below
[]
Now loading selected_solar.csv
scaling now
interpolate should be below
[]
Now loading selected_solar.csv
scaling now
interpolate should be below
[]
Now loading selected_solar.csv
scaling now
Using Wandb config:
{'batch_size': 2, 'forecast_history': 1, 'lr': 0.001, 'out_seq_length': 1, 'model_name': 'MultiAttnHeadSimple', 'model_type': 'PyTorch', 'model_params': {'number_time_series': 3, 'seq_len': 1, 'output_seq_len': 1, 'forecast_length': 1}, 'weight_path_add': {'excluded_layers': ['last_layer.weight', 'last_layer.bias']}, 'dataset_params': {'class': 'default', 'training_path': 'selected_solar.csv', 'validation_path': 'selected_solar.csv', 'test_path': 'selected_solar.csv', 'batch_size': 2, 'forecast_history': 1, 'forecast_length': 1, 'train_end': 3049, 'valid_start': 3050, 'valid_end': 3920, 'target_col': ['Power(MW)'], 'relevant_cols': ['Power(MW)', 'month', 'weekday'], 'scaler': 'StandardScaler', '

epoch,9
_runtime,110
_timestamp,1629138867
_step,20
loss,0.17097


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
_runtime,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▆▆▇▇█
_timestamp,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▆▆▇▇█
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
loss,█▂▂▁▂▁▁▁▁▁


...

Run 1392uszc errored: IndexError('index 0 is out of bounds for axis 0 with size 0')
wandb: ERROR Run 1392uszc errored: IndexError('index 0 is out of bounds for axis 0 with size 0')
wandb: Agent Starting Run: jelyasez with config:
wandb: 	batch_size: 2
wandb: 	forecast_history: 1
wandb: 	lr: 0.01
wandb: 	out_seq_length: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


sucessfully deleted layers
Weights sucessfully loaded
interpolate should be below
[]
Now loading selected_solar.csv
scaling now
interpolate should be below
[]
Now loading selected_solar.csv
scaling now
interpolate should be below
[]
Now loading selected_solar.csv
scaling now
Using Wandb config:
{'batch_size': 2, 'forecast_history': 1, 'lr': 0.01, 'out_seq_length': 3, 'model_name': 'MultiAttnHeadSimple', 'model_type': 'PyTorch', 'model_params': {'number_time_series': 3, 'seq_len': 1, 'output_seq_len': 3, 'forecast_length': 3}, 'weight_path_add': {'excluded_layers': ['last_layer.weight', 'last_layer.bias']}, 'dataset_params': {'class': 'default', 'training_path': 'selected_solar.csv', 'validation_path': 'selected_solar.csv', 'test_path': 'selected_solar.csv', 'batch_size': 2, 'forecast_history': 1, 'forecast_length': 3, 'train_end': 3049, 'valid_start': 3050, 'valid_end': 3920, 'target_col': ['Power(MW)'], 'relevant_cols': ['Power(MW)', 'month', 'weekday'], 'scaler': 'StandardScaler', 'i

epoch,9
_runtime,111
_timestamp,1629139457
_step,20
loss,0.42571


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
_runtime,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▆▆▇▇█
_timestamp,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▆▆▇▇█
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
loss,█▄▃▁▁▁▁▁▁▁


Run jelyasez errored: IndexError('index 0 is out of bounds for axis 0 with size 0')
wandb: ERROR Run jelyasez errored: IndexError('index 0 is out of bounds for axis 0 with size 0')
Detected 5 failed runs in a row at start, killing sweep.
wandb: ERROR Detected 5 failed runs in a row at start, killing sweep.
wandb: To change this value set WANDB_AGENT_MAX_INITIAL_FAILURES=val
